In [1]:
from papermill import execute_notebook, PapermillExecutionError
import pandas as pd
import pathlib
import numpy as np
from scipy.sparse import csr_matrix, vstack
import anndata

## Runner

In [4]:
dmr_dir = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_DissectionRegion/'
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_DissectionRegion/ITSpatial_rms_results_collapsed.tsv'

skip_quantile = 0.25
delta_to_mean = 0.3
dms_cutoff = 2

black_list_path = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'
motif_file_path = '/home/hanliu/ref/MEME/selected_motifs_sets/JASPAR2020_CORE_vertebrates_non-redundant_pfms.has_mouse_id.meme'
genome_fasta_path = '/home/hanliu/ref/mouse/genome/fasta/raw/mm10.fa'
chrom_size_path = '/home/hanliu/ref/mouse/genome/mm10.main.chrom.sizes'

In [5]:
# set up 
dmr_dir = pathlib.Path(dmr_dir)
recipe_dir = dmr_dir / 'Recipe'
recipe_dir.mkdir(exist_ok=True)

In [7]:
# step 1 filter 
params = dict(dmr_path=dmr_path,
              dmr_prefix='ITSpatial',
              black_list_path=black_list_path,
              skip_quantile=skip_quantile,
              delta_to_mean=delta_to_mean,
              dms_cutoff=dms_cutoff)

input_path = '1.FilterDMRRecipe_robust_mean_delta.ipynb'
output_path = recipe_dir / '1.FilterDMRRecipe_robust_mean_delta.ipynb'
# check the last file generated by this notebook
if not (dmr_dir / 'HyperDMR/TotalHits.h5ad').exists():
    execute_notebook(str(input_path),
                     str(output_path),
                     parameters=params,
                     engine_name=None,
                     prepare_only=False,
                     kernel_name=None,
                     progress_bar=True,
                     log_output=False,
                     start_timeout=60,
                     report_mode=False,
                     cwd=str(recipe_dir))

In [8]:
# step 2 motif scan of total DMR
params = dict(bed_path=str(dmr_dir / 'TotalDMR.nofilter.bed'),
              motif_file_path=motif_file_path,
              genome_fasta_path=genome_fasta_path,
              chrom_size_path=chrom_size_path)

if not (dmr_dir / 'MotifScan.h5ad').exists():
    input_path = '2.MotifScan.ipynb'
    output_path = recipe_dir / '2.MotifScan.ipynb'
    execute_notebook(str(input_path),
                     str(output_path),
                     parameters=params,
                     engine_name=None,
                     prepare_only=False,
                     kernel_name=None,
                     progress_bar=True,
                     log_output=False,
                     start_timeout=60,
                     report_mode=False,
                     cwd=str(recipe_dir))


In [9]:


# step 3 
params = dict(dmr_bed=str(dmr_dir / 'TotalDMR.nofilter.bed'))
if not (dmr_dir / 'Annotation/DMRAnnotation.h5ad').exists():
    input_path = '3.DMRAnnotation.ipynb'
    output_path = recipe_dir / '3.DMRAnnotation.ipynb'
    execute_notebook(str(input_path),
                     str(output_path),
                     parameters=params,
                     engine_name=None,
                     prepare_only=False,
                     kernel_name=None,
                     progress_bar=True,
                     log_output=False,
                     start_timeout=60,
                     report_mode=False,
                     cwd=str(recipe_dir))
return

PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [7]":
---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
<ipython-input-7-a8e71617b18e> in <module>
     12     for fname, path in path_dict.items():
     13         print(name, fname)
---> 14         this_bed = BedTool(str(path))
     15         dmr_bed.intersect(this_bed, wa=True, u=True).saveas(f'{_output_dir}/{fname}.bed')

~/miniconda3/envs/clustering/lib/python3.7/site-packages/pybedtools/bedtool.py in __init__(self, fn, from_string, remote)
    460                         if six.PY2:
    461                             raise ValueError(msg)
--> 462                         raise FileNotFoundError(msg)
    463                     self._isbam = isBAM(fn)
    464 

FileNotFoundError: File "/home/hanliu/ref/inhouse/He_2020_Nature_Mouse_Tissue_Developmental